# Описание идеи
Идея в том, чтобы обучив сеть на некотором фиксированном наборе авторов, отрезать от неё последний слой и  поставить на его места классификатор типа "K ближайших соседей"(KNN). 

То есть KNN будет работать на признаках с предпоследнего слоя сети.

Т.к. KNN можно мгновенно завести на новом наборе классов мы добьёмся того, что, по сути, без переобучения сможем использовать модель на новых авторах.
Собственно  в этом ноуте попытка это сделать для простой задачи классификации, где можно "ввернуть" нейронки - MNIST.

1. Берём исходный набор данных и выкидываем все сэмплы с цифрами `8` и `9`.
2. Обучаем сетку с `N`-ым количеством слоёв  и обучаем на урезанном наборе, добиваясь приемлимого качества.
3. Выкидываем `N`-ый слой.
4. Прогоняем обученную сеть на всём тренировочном и наборе для текстов, извлекая признаки с `N-1`-го слоя.
5. Обучаем KNN на тренировчном наборе с признаками сети.
6. Оцениваем качество классификации KNN на двух подмножествах тестового набора:
    - подмножество сэмплов с только лишь новыми классами(цифры `8` и `9`)
    - подмножество сэмплов с классами, на которых ранее обучалась сеть ( цифры от `0` до `7`).

[Результаты.](#Результаты)

In [2]:
%pylab inline
import numpy
import pandas

Populating the interactive namespace from numpy and matplotlib


In [3]:
from keras.datasets import mnist
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# # load dataset
# dataframe = pandas.read_csv("iris.data.txt", header=None)
# dataset = dataframe.values
# X = dataset[:,0:4].astype(float)
# Y = dataset[:,4]
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
print(np.unique(y_train))

y_enc = np_utils.to_categorical(encoder.transform(y_train))
y_test_enc = np_utils.to_categorical(encoder.transform(y_test))

/home/grigory/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/grigory/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[0 1 2 3 4 5 6 7 8 9]


In [4]:
cut_mask = y_enc[:,-2:].sum(axis=1)==0
y_enc_cut = y_enc[cut_mask, :8]
x_cut_train = x_train[cut_mask,:]
x_train.shape, x_cut_train.shape

((60000, 28, 28), (48200, 28, 28))

# Обучаем KNN на исходных данных

Update:
> не долждался пока knn отрабоатет на исходных данных высокой размерности.

In [10]:
from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(4, n_jobs=-1)
# knn.fit(x_train.reshape(-1,28*28), y_enc)

In [11]:
# %%time
# knn.score(x_test.reshape(-1,28*28), y_test_enc)

# Обучаем сеть на исходных данных

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense,Flatten, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import optimizers

import gc
gc.collect()
# create model
model = Sequential()
model.add(Dense(40, input_shape=(28,28), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(8, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])
model.fit(x_cut_train, y_enc_cut, 
          epochs=10, 
          validation_split=0.3,  
          verbose=1);
model.summary()

Train on 33740 samples, validate on 14460 samples
Epoch 1/10
33740/33740 [==============================] - 7s 195us/step - loss: 5.1147 - acc: 0.6311 - val_loss: 1.4306 - val_acc: 0.8761

In [14]:
intermediate_layer_model = Model(inputs = model.input, outputs = model.layers[-2].output)
features_train = intermediate_layer_model.predict(x_train)
features_test = intermediate_layer_model.predict(x_test)

# Обучаем KNN на признаках с предпоследнего слоя сети

In [ ]:
knn = KNeighborsClassifier(4)
knn.fit(features_train, y_enc)

# Результаты

In [15]:
# выберем только примеры с последними цифрами в наборе(8 и 9)
test_cut_mask = y_test_enc[:,-2:].sum(axis=1)==0

print('Точность исключительно на принципиально новых данных(новые цифры в наборе):{:.3f}'.format(
    knn.score(features_test[~test_cut_mask,:], y_test_enc[~test_cut_mask])))

test_cut_mask = y_test_enc[:,-2:].sum(axis=1)==0
print('Точность на старых данных(старый набор цифр):{:.3f}'.format(
    knn.score(features_test[test_cut_mask,:], y_test_enc[test_cut_mask])))

Точность исключительно на принципиально новых данных(новые цифры в наборе):0.799
Точность на старых данных(старый набор цифр):0.925


> Вывод:
**качество просело, но всё равно гораздо лучше случайного гадания.
И это пока без подбора параметров.**